# IPO Intelligence Database Explorer 🗄️

This notebook connects directly to the PostgreSQL database and allows you to explore the data.

## Database Schema
- **documents** - Document metadata
- **chapters** - Chapter information
- **chunks** - Text chunks from documents
- **embeddings** - Vector embeddings (384-dim)

In [ ]:
# Setup - Run this first
import sys
sys.path.insert(0, './src')

from database.connection import get_db, test_connection
from database.repositories import DocumentRepository, ChunkRepository, EmbeddingRepository
from sqlalchemy import text
import pandas as pd
import numpy as np

print("✅ Imports successful")
test_connection()

: 

## 1. View All Documents

In [ ]:
# Get all documents
docs = DocumentRepository.get_all()

# Convert to DataFrame for better display
df_docs = pd.DataFrame(docs)
df_docs[['document_id', 'display_name', 'total_pages', 'total_chunks', 'upload_date']]

## 2. Query Documents Table Directly

In [ ]:
# Raw SQL query
with get_db() as db:
    query = text("""
        SELECT 
            document_id,
            display_name,
            total_pages,
            total_chunks,
            file_hash,
            upload_date
        FROM documents
        ORDER BY created_at DESC
    """)
    result = db.execute(query)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())

df

## 3. Explore Chunks for a Document

In [ ]:
# Change this to explore different documents
DOCUMENT_ID = 'pw_ipo'  # or 'policybazar_ipo', 'emt_ipo', 'lenskart_solutions_limited_drhp_1753782641'

# Get chunks
chunks = ChunkRepository.get_by_document(DOCUMENT_ID)

print(f"📦 Found {len(chunks)} chunks for {DOCUMENT_ID}\n")
print("First 5 chunks:")
pd.DataFrame(chunks[:5])[['chunk_index', 'page_number', 'word_count', 'text']].head()

## 4. Database Statistics

In [ ]:
with get_db() as db:
    stats_query = text("""
        SELECT 
            d.document_id,
            d.display_name,
            COUNT(DISTINCT c.id) as chunk_count,
            COUNT(DISTINCT e.id) as embedding_count,
            AVG(c.word_count) as avg_chunk_words
        FROM documents d
        LEFT JOIN chunks c ON c.document_id = d.id
        LEFT JOIN embeddings e ON e.chunk_id = c.id
        GROUP BY d.id
        ORDER BY d.created_at DESC
    """)
    result = db.execute(stats_query)
    stats_df = pd.DataFrame(result.fetchall(), columns=result.keys())

stats_df

## 5. Test Vector Similarity Search

In [ ]:
# Create a random query vector
query_vector = np.random.rand(384).tolist()

# Search for similar chunks
results = EmbeddingRepository.search_similar(
    query_embedding=query_vector,
    document_id='pw_ipo',
    top_k=5
)

print(f"🔍 Found {len(results)} similar chunks:\n")
for i, r in enumerate(results, 1):
    print(f"{i}. Similarity: {r['similarity']:.4f}")
    print(f"   Page: {r['page_number']}")
    print(f"   Text: {r['text'][:100]}...\n")

## 6. Search Chunks by Text

In [ ]:
# Search for chunks containing specific text
search_term = 'revenue'  # Change this to search for different terms

with get_db() as db:
    search_query = text("""
        SELECT 
            d.display_name,
            c.chunk_index,
            c.page_number,
            c.text
        FROM chunks c
        JOIN documents d ON d.id = c.document_id
        WHERE LOWER(c.text) LIKE :search_term
        LIMIT 10
    """)
    result = db.execute(search_query, {'search_term': f'%{search_term.lower()}%'})
    search_df = pd.DataFrame(result.fetchall(), columns=result.keys())

print(f"Found {len(search_df)} chunks containing '{search_term}':\n")
search_df

## 7. Custom SQL Queries

Write your own SQL queries here:

In [ ]:
# Example: Find longest chunks
with get_db() as db:
    custom_query = text("""
        SELECT 
            d.display_name,
            c.chunk_index,
            c.page_number,
            c.word_count,
            LEFT(c.text, 100) as text_preview
        FROM chunks c
        JOIN documents d ON d.id = c.document_id
        ORDER BY c.word_count DESC
        LIMIT 10
    """)
    result = db.execute(custom_query)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())

df

## 8. Database Health Check

In [ ]:
with get_db() as db:
    # Get table sizes
    health_query = text("""
        SELECT 
            'documents' as table_name,
            COUNT(*) as row_count
        FROM documents
        
        UNION ALL
        
        SELECT 
            'chapters' as table_name,
            COUNT(*) as row_count
        FROM chapters
        
        UNION ALL
        
        SELECT 
            'chunks' as table_name,
            COUNT(*) as row_count
        FROM chunks
        
        UNION ALL
        
        SELECT 
            'embeddings' as table_name,
            COUNT(*) as row_count
        FROM embeddings
    """)
    result = db.execute(health_query)
    health_df = pd.DataFrame(result.fetchall(), columns=result.keys())

print("📊 Database Health Status:\n")
health_df

## 9. Export Data

Export query results to CSV:

In [ ]:
# Export documents to CSV
df_docs.to_csv('documents_export.csv', index=False)
print("✅ Exported to documents_export.csv")